# Example of using Delta Lake 0.4 without Databricks

[Delta Lake](https://delta.io/) is an open-source storage layer that brings ACID transactions to Apache Spark™ and big data workloads. But it is **way more** then that!

This is a rewritten notebook example from this [blog post](https://databricks.com/blog/2019/10/03/simple-reliable-upserts-and-deletes-on-delta-lake-tables-using-python-apis.html) by Databricks. The intension is to show why Delta Lake is a big deal and how to run Delta Lake without a Databricks services.

Delta Lake examples in this notebook:
* Convert data to as Delta Lake format
* Create Delta Lake table
* Spark SQL capabilities
* Delete data
* Update data
* View audit history of table
* Merge (union) of two tables which remove duplicates, updates rows and add a new row

For testing this docker can be used: ```docker run -it --rm -p 8888:8888 -p 4040:4040 jupyter/pyspark-notebook```

### Author
Anders Boje Larsen - [alarsen@deloitte.dk](alarsen@deloitte.dk) - [LinkedIn](https://www.linkedin.com/in/andersboje/)

### Data Preparation
Configure locations for the source file and where the Delta Lake Table will be stored

In [ ]:
tripdelaysFilePath = "departuredelays.csv" 
pathToEventsTable = "departureDelays.delta"
flightdata = "https://raw.githubusercontent.com/drabastomek/learningPySpark/master/Chapter03/flight-data/departuredelays.csv"

In [ ]:
!pip install --upgrade pyspark

In [ ]:
#Download data flight dataset
!rm -fr departureDelays.delta
!wget https://raw.githubusercontent.com/drabastomek/learningPySpark/master/Chapter03/flight-data/departuredelays.csv

In [ ]:
import pandas as pd
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext, SparkConf
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.11:0.4.0 pyspark-shell'

sc_conf = SparkConf()
sc_conf.set('spark.databricks.delta.retentionDurationCheck.enabled', 'false')
sc_conf.set('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
print(sc_conf.getAll())

try:
    sc.stop()
    sc = SparkContext(conf=sc_conf)
except:
    sc = SparkContext(conf=sc_conf)

spark = SparkSession(sc)

Create `departureDelays` DataFrame

In [ ]:
departureDelays = spark.read.option("header", "true").option("inferSchema", "true").csv(tripdelaysFilePath)

Save table as Delta Lake (update `pathToEventsTable` to match the following location

In [ ]:
departureDelays.write.format("delta").mode("overwrite").save(pathToEventsTable)

Load Delta Lake table

In [ ]:
delays_delta = spark.read.format("delta").load(pathToEventsTable)
delays_delta.createOrReplaceTempView("delays_delta")

Get count of rows

In [ ]:
spark.sql("select count(1) from delays_delta where origin = 'SEA' and destination = 'SFO'").toPandas()

**Review File System**: Note there are four files initially created as part of the table creation.

In [ ]:
%ls $pathToEventsTable

### Deletes
With Delta Lake, you can delete data with the Python API

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *
deltaTable = DeltaTable.forPath(spark, pathToEventsTable)
deltaTable.delete("delay < 0") 

In [ ]:
# Get Row Count
spark.sql("select count(1) from delays_delta where origin = 'SEA' and destination = 'SFO'").toPandas()

**Review File System**: Note that while we deleted early (and on-time) flights, there are now eight files (instead of the four files initially created as part of the table creation).

In [ ]:
%ls $pathToEventsTable

### Updates
Update flights originating from Detroit (DTW) to now be from Seattle (SEA)

In [ ]:
deltaTable.update("origin = 'DTW'", { "origin": "'SEA'" } ) 

In [ ]:
spark.sql("select count(1) from delays_delta where origin = 'SEA' and destination = 'SFO'").toPandas()

### View History
View the table history (note the create table, insert, and update operations)

In [ ]:
deltaTable.history().toPandas()

Calculate counts for each version of the table

In [ ]:
dfv0 = spark.read.format("delta").option("versionAsOf", 0).load("departureDelays.delta")
dfv1 = spark.read.format("delta").option("versionAsOf", 1).load("departureDelays.delta")
dfv2 = spark.read.format("delta").option("versionAsOf", 2).load("departureDelays.delta")

cnt0 = dfv0.where("origin = 'SEA'").where("destination = 'SFO'").count()
cnt1 = dfv1.where("origin = 'SEA'").where("destination = 'SFO'").count()
cnt2 = dfv2.where("origin = 'SEA'").where("destination = 'SFO'").count()

print("SEA -> SFO Counts: Create Table: %s, Delete: %s, Update: %s" % (cnt0, cnt1, cnt2))

**Review File System**: Note the number of files based on the preceding operations.

In [ ]:
%ls $pathToEventsTable

### Vacuum
Remove older data (by default 7 days) 

In [ ]:
deltaTable.vacuum(0)

In [ ]:
%ls $pathToEventsTable

And let's not forget, Delta Lake 0.4.0 also includes `MERGE` in the Python API!

### Merge
Let's merge another table with the `departureDelays` table with [data deduplication](https://docs.delta.io/0.4.0/delta-update.html#data-deduplication-when-writing-into-delta-tables).  Let's start by viewing data that will be impacted by the merge.

In [ ]:
spark.sql("select * from delays_delta where origin = 'SEA' and destination = 'SFO' and date like '1010%' order by date limit 10").toPandas()

Next, let's create our `merge_table` which contains three rows:
* 1010710: this row is a duplicate
* 1010521: this row will be updated with a new delay value
* 1010822: this is a new row

In [ ]:
items = [(1010521, 10, 590, 'SEA', 'SFO'), (1010710, 31, 590, 'SEA', 'SFO'), (1010832, 31, 590, 'SEA', 'SFO')]
cols = ['date', 'delay', 'distance', 'origin', 'destination']
merge_table = spark.createDataFrame(items, cols)
merge_table.toPandas()

Let's run our merge statement that will handle the duplicates, updates, and add a new row

In [ ]:
deltaTable.alias("flights") \
    .merge(merge_table.alias("updates"),"flights.date = updates.date") \
    .whenMatchedUpdate(set = { "delay" : "updates.delay" } ) \
    .whenNotMatchedInsertAll() \
    .execute()

In [ ]:
spark.sql("select * from delays_delta where origin = 'SEA' and destination = 'SFO' and date like '1010%' order by date limit 10").toPandas()

As noted in the previous cells, notice the following:
* There is only one row for the date `1010710` as `merge` automatically takes care of **data deduplication**
* The row for the date `1010521` has the `delay` value **updated** from 0 to 10.
* The row for the date `1010821` has been added as this date did not exist, hence it was **inserted**.